In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res

In [3]:
### 目标函数：

bounds = [(-10,10),(-10,10),(-10,10),(-10,10),(-10,10),(-10,10),(-10,10),(-10,10),(0,20)]
X_optimal = [-0.657776192427943163, -0.153418773482438542, 0.323413871675240938, -0.946257611651304398,-0.657776194376798906, -0.753213434632691414, 0.323413874123576972, -0.346462947962331735,0.59979466285217542]
y_optimal = -0.866025403784439
def G18(X):
    y = -0.5*(X[0]*X[3]-X[1]*X[2]+X[2]*X[8]-X[4]*X[8]+X[4]*X[7]-X[5]*X[6])
    g1 = X[2]**2+X[3]**2-1
    g2 = X[8]**2-1
    g3 = X[4]**2+X[5]**2-1
    g4 = X[0]**2+(X[1]-X[8])**2-1
    g5 = (X[0]-X[4])**2+(X[1]-X[5])**2-1
    g6 = (X[0]-X[6])**2+(X[1]-X[7])**2-1
    g7 = (X[2]-X[4])**2+(X[3]-X[5])**2-1
    g8 = (X[2]-X[6])**2+(X[3]-X[7])**2-1
    g9 = X[6]**2+(X[7]-X[8])**2-1
    g10 = X[1]*X[2]-X[0]*X[3]
    g11 = -X[2]*X[8]
    g12 = X[4]*X[8]
    g13 = X[5]*X[6]-X[4]*X[7]
    return y,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13

In [4]:
### 适应度函数
def fitnessG18_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1.0
    fmin = -1.0
    epsion = 100
    count = np.sum(np.array(G18(X))[1:14]>0)
    
    return -(G18(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG18_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1.0
    fmin = -1.0
    vmax = 6791
    e = 0.0001
    n = 13
    
    res = G18(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:14]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness  

In [5]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+28)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+28)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG18_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG18_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [6]:
PSO = PSO_fit(pN=4000, dim=9, max_iter=1000, bounds=bounds)

In [7]:
PSO.init_Population()

In [8]:
gbest, gfit = PSO.iterator()

[-1103.85014785 -1080.62542575  -596.05874356 ... -1184.30930111
  -766.10565585  -826.24792863]
[-281.74567487 -495.14776956 -556.73755279 ... -557.5201582  -226.94765761
 -826.24792863]
[-281.74567487 -281.74567487 -451.29082906 ... -557.5201582  -226.94765761
 -583.73142095]
[-196.45654168 -214.37855035 -180.1880403  ... -366.1786303  -226.94765761
 -272.03912067]
[-196.45654168 -214.37855035 -180.1880403  ... -366.1786303  -226.94765761
 -246.39279788]
[-178.67428482 -194.41901891 -180.1880403  ... -149.36633576 -226.94765761
 -246.39279788]
[-178.67428482 -194.41901891 -177.78192865 ... -149.36633576 -149.36633576
 -189.47188265]
[-165.59161362 -194.41901891 -177.78192865 ... -107.10859827  -90.39769368
 -133.09625799]
[ -51.23360036 -166.60146161 -143.83766926 ... -107.10859827  -60.87387851
  -92.57678756]
[ -51.23360036 -145.44347109  -88.21546874 ...  -78.13775295  -60.87387851
  -92.57678756]
[-42.22529068 -30.16346543 -88.21546874 ... -60.02551348 -46.98484032
 -28.47075941]

[0.84640634 0.84707773 0.85155091 ... 0.8491356  0.84513794 0.85155091]
[0.84732947 0.84707773 0.85155091 ... 0.8491356  0.84513794 0.85155091]
[0.84732947 0.84707773 0.85739148 ... 0.8491356  0.84513794 0.85248735]
[0.84732947 0.85109597 0.85739148 ... 0.8491356  0.84513794 0.85274621]
[0.84732947 0.85109597 0.85739148 ... 0.85146624 0.85053336 0.85274621]
[0.84879885 0.85554554 0.85739148 ... 0.85146624 0.85053336 0.85274621]
[0.84879885 0.85554554 0.85739148 ... 0.85182603 0.85581553 0.85274621]
[0.85145207 0.85738636 0.85739148 ... 0.85449235 0.85581553 0.85274621]
[0.85145207 0.85738636 0.85739148 ... 0.85700215 0.85581553 0.85643144]
[0.85780034 0.85738636 0.85739148 ... 0.85700215 0.85581553 0.85643144]
[0.85780034 0.86009679 0.85739148 ... 0.85700215 0.85590024 0.86254571]
[0.8599561  0.86009679 0.85739148 ... 0.86124032 0.85738636 0.86254571]
[0.8599561  0.86009679 0.85739148 ... 0.86124032 0.85738636 0.86254571]
[0.8599561  0.86009679 0.85739148 ... 0.86124032 0.85839286 0.86

[0.86601138 0.86601006 0.86601167 ... 0.86601165 0.86601521 0.8660145 ]
[0.86601138 0.86601006 0.86601383 ... 0.86601425 0.86601521 0.8660145 ]
[0.86601336 0.86601818 0.86601383 ... 0.86601425 0.86601521 0.8660145 ]
[0.86601818 0.86601818 0.86601383 ... 0.86601425 0.8660156  0.86601763]
[0.86601818 0.86601818 0.86601383 ... 0.86601612 0.86601832 0.86601763]
[0.86601818 0.86601818 0.86601383 ... 0.86601763 0.86601832 0.86601763]
[0.86601818 0.8660196  0.86601828 ... 0.86601763 0.86602014 0.86601763]
[0.86601818 0.8660196  0.86601835 ... 0.86601763 0.86602014 0.86601763]
[0.86601818 0.8660196  0.86601835 ... 0.86602041 0.86602014 0.86601816]
[0.86601832 0.8660196  0.86601876 ... 0.86602041 0.86602108 0.86601921]
[0.8660194  0.8660196  0.86602049 ... 0.86602041 0.86602108 0.86601921]
[0.86602105 0.8660196  0.86602049 ... 0.86602041 0.86602108 0.86602092]
[0.86602106 0.86602054 0.8660209  ... 0.86602041 0.86602108 0.86602092]
[0.86602106 0.86602054 0.86602144 ... 0.86602041 0.86602108 0.86

[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254

[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254

[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254

[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254

[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254

[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254 0.8660254 0.8660254 ... 0.8660254 0.8660254 0.8660254]
[0.8660254

In [9]:
G18(gbest)

(array([ 1.26016747e-09, -1.26016753e-09, -3.15693932e-09,  3.15693921e-09,
         2.52380927e-09, -2.52380926e-09, -1.57629665e-09,  1.57629676e-09,
        -0.00000000e+00]),
 array([ 0.65508372, -0.65508372,  0.32678908, -0.32678908,  0.65508373,
        -0.65508373,  0.32678909, -0.32678909, -1.        ]),
 array([-0.17245814, -0.82754186, -0.33660546, -0.66339454, -0.17245814,
        -0.82754186, -0.33660546, -0.66339454, -1.        ]),
 array([ 0.65508373, -0.65508373,  0.32678908, -0.32678908,  0.65508373,
        -0.65508373,  0.32678908, -0.32678908, -1.        ]),
 array([-0.17245814, -0.82754186, -0.33660546, -0.66339454, -0.17245814,
        -0.82754186, -0.33660546, -0.66339454, -1.        ]),
 array([-1., -1., -1., -1., -1., -1., -1., -1., -1.]),
 array([-1., -1., -1., -1., -1., -1., -1., -1., -1.]),
 array([-1., -1., -1., -1., -1., -1., -1., -1., -1.]),
 array([-1., -1., -1., -1., -1., -1., -1., -1., -1.]),
 array([-0.17245814, -0.82754186, -0.33660546, -0.66339454, -

In [10]:
G18(gbest[0])

(-0.8660254037844388,
 0.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 -0.8660254035020734,
 0.0,
 -0.0,
 -0.8660254040668041)